In [1]:
## 라이브러리 설정
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import re

import random

import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
## 시각화 설정
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
style.use('ggplot')
%matplotlib inline

sns.set()
sns.set_style('whitegrid')
sns.set_color_codes()

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname = path, size = 20).get_name()
rc('font', family = font_name)
plt.rcParams['axes.unicode_minus'] = False

In [44]:
from time import sleep

unique_word = set()
def crawling_related_word(queue, n):
    global unique_word
    if len(queue)==0 or n==0:
        print('크롤링 종료')
        return
    df = pd.DataFrame()
    next_queue = []
    print(queue)

    for word in queue:
        temp_df = pd.DataFrame()
        temp_list = []
        url = 'https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=' + word

        while True:
            try:
                response = requests.get(url)
                break
            except:
                print("sleeping...")
                sleep(5)

        if response.status_code == 200:
            html = response.text
            document = bs(html, 'html.parser')
            try:
                div_inner = document.find('div',class_='list_keyword type2')
                related_word_list = div_inner.find_all('span', class_='wsn')
            except:
                if pd.isna(div_inner):
                    try:
                        div_inner = document.find('div', class_='list_keyword')
                        related_word_list = div_inner.find_all('span',class_='wsn')
                    except:
                        print(word,'연관 검색어가 없습니다.')
                        related_word_list = []

        if len(related_word_list) == 0: continue
        
        for idx, related_word in enumerate(related_word_list):
            next_word = related_word_list[idx].text
            temp_list.append(next_word)
            if not next_word in unique_word:
                next_queue.append(next_word)
                unique_word.add(next_word)
            # print(related_word_list[idx].text)
            
        temp_df['sub_word'] = temp_list
        temp_df['sup_word'] = word
        df = pd.concat([df,temp_df],axis=0)
    df = pd.concat([df, crawling_related_word(next_queue, n-1)], axis=0)
    return df.reset_index(drop=True)

word_list = ['코로나 확진자', '코로나19']
df_crawl = crawling_related_word(word_list,3)

['코로나 확진자', '코로나19']
코로나 확진자 연관 검색어가 없습니다.
['코로나19 확진자 현황', '코로나19 거리두기', '코로나19 세계 현황', '코로나19 단계', '코로나19 백신', '코로나19 증상', '코로나19 뜻', '코로나19 일본', '코로나19 치료제', '위드 코로나', '코로나19 예방수칙', '코로나19 대구', '코로나19 부산', '코로나19 3단계', '코로나19 광주', '코로나19 사망자', '코로나19 미국', '코로나19 2단계', '코로나 19 서울', '코로나19 바이러스']
['세계 코로나19 확진자 현황', '전국 코로나19 확진자 현황', '코로나19 확진자 현황 그래프', '코로나19 신규 확진자 현황', '코로나19 확진자 추이', '국내 코로나19 확진자 현황', '일본 코로나19 확진자 현황', '코로나 19 확진자 수', '금일 코로나19 확진자 현황', '코로나19 확진자 일일현황', '인천 코로나19 확진자 현황', '부산 코로나19 확진자 현황', '미국 코로나19 확진자 현황', '코로나19 확진자 지역별 현황', '코로나19 추가 확진자 현황', '오늘 코로나19 신규 확진자 현황', '코로나19 확진자 동선', '경기도 코로나19 확진자 현황', '코로나19 거리두기 단계', '코로나19 거리두기 발표', '코로나19 거리두기 연장', '코로나 19 사회적 거리두기', '코로나19 거리두기 완화', '코로나19 거리두기 3단계', '코로나19 거리두기 4단계', '코로나19 거리두기 조정', '코로나19 사회적 거리두기 단계', '코로나19 거리두기 2단계', '코로나19 거리두기 5단계', '코로나19 거리두기 1단계', '코로나 19 거리두기1.5단계', '코로나19 사회적 거리두기 2단계', '코로나19 거리두기 4단계 연장', '코로나 4단계 조치', '대전 코로나19 거리두기', '코로나19 거리두기 개편', '코로나 거리두기 결혼식', '10월 18일 거리두기', '코로나

In [45]:
df_crawl

,sub_word,sup_word
0,코로나19 확진자 현황,코로나19
1,코로나19 거리두기,코로나19
2,코로나19 세계 현황,코로나19
3,코로나19 단계,코로나19
4,코로나19 백신,코로나19
...,...,...
5032,휴온스 이오패치,휴온스
5033,휴온스 네이처,휴온스
5034,이수앱지스,휴온스
5035,휴온스 진단키트,휴온스
